In [7]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
pd.set_option("display.max_colwidth", 0)
# from IPython.core.display import display, HTML
# display(HTML("<style>div.output_scroll { height: 100em; }</style>"))


In [2]:
raw_data = pd.read_csv("play_sessions.csv")

In [3]:
len(raw_data)

953

In [4]:
raw_data = raw_data[raw_data.user_id.notnull()]
raw_data = raw_data[raw_data.version == "1.0.3"]
raw_data = raw_data.reset_index()
len(raw_data)

619

![diagram](diagram.png)

In [5]:
class Episode():
    def __init__(self):
        self.passing = False
        self.programming_interface = pd.DataFrame()
        self.episode_data = pd.DataFrame()
    def __init__(self, pi:pd.DataFrame, ed:pd.DataFrame, passing:bool):
        self.passing = passing
        self.programming_interface = pi
        self.episode_data = ed
        

In [6]:
def parse_raw_data_frames(row:int) -> pd.DataFrame:
    frames = raw_data.frames[row]
    obj = json.loads(frames)
    for i, o in enumerate(obj):
        obj[i] = json.loads(obj[i])
    session = pd.DataFrame(obj)
    return session

memo_frames = {}

def iter_session_frames():
    for i in raw_data.index:
        if i not in memo_frames:   
            frames = parse_raw_data_frames(i)
            memo_frames[i] = frames
        yield memo_frames[i].copy()

def iter_enum_session_frames(): #TODO: uh make this not a weird copy
    for i in raw_data.index:
        yield i, parse_raw_data_frames(i)

In [ ]:
organized_sessions = defaultdict(list)
other_actors = pd.DataFrame()

for i, all_frames in iter_enum_session_frames():
    if len(all_frames) == 0:
        continue
    user_id = raw_data.user_id[i]
    
    episode_list = []
    passing = False
    curr_prog_interface = pd.DataFrame(columns=all_frames.columns)
    prev_prog_interface = pd.DataFrame(columns=all_frames.columns)
    curr_episode_data = pd.DataFrame(columns=all_frames.columns)
    for i, frame in all_frames.iterrows():
        if frame.actor == "episode_data":
            if frame.object_name == "challenge_pass":
                passing = True
            if frame.verb == "episode_started":
                episode_list.append(Episode(passing=passing, pi=prev_prog_interface, ed=curr_episode_data))
                passing = False
                prev_prog_interface = curr_prog_interface
                curr_prog_interface = pd.DataFrame(columns=all_frames.columns)
                curr_episode_data = pd.DataFrame(columns=all_frames.columns)
            curr_episode_data.loc[i] = frame
        elif frame.actor == "programming_interface":
            curr_prog_interface.loc[i] = frame
        else:
            other_actors = pd.concat([other_actors, frame.to_frame(1).T],  ignore_index=True, sort=True)
    # record last episode and last program changes
    episode_list.append(Episode(passing=passing, pi=prev_prog_interface, ed=curr_episode_data))
    if len(curr_prog_interface) > 0:
        episode_list.append(Episode(passing=False, pi=curr_prog_interface, ed=pd.DataFrame(columns=all_frames.columns)))
        
    organized_sessions[user_id].append(episode_list)
    

In [18]:
other_actors

actor assent consent               context  \
0     cmra-blocks           True   True    {'version': '1.0.3'}   
1     curriculum_interface  NaN    NaN     None                   
2     curriculum_interface  NaN    NaN     None                   
3     curriculum_interface  NaN    NaN     None                   
4     cmra-blocks           True   True    {'version': '1.0.3'}   
...           ...            ...    ...                     ...   
1767  curriculum_interface  NaN    NaN     None                   
1768  cmra-blocks           True   True    {'version': '1.0.3'}   
1769  curriculum_interface  NaN    NaN     None                   
1770  cmra-blocks           True   True    {'version': '1.0.3'}   
1771  curriculum_interface  NaN    NaN     None                   

                        domain  fps object_name  \
0     cmra-blocks-dev.cs2n.org  NaN  SPIKE        
1     NaN                       NaN  0            
2     NaN                       NaN  1            
3     NaN                       NaN  2            
4     cmra-blocks-dev.cs2n.org  NaN  SPIKE        
...                        ...  ...    ...        
1767  NaN                       NaN  0            
1768  cmra-blocks.cs2n.org      NaN  SPIKE        
1769  NaN                       NaN  0            
1770  cmra-blocks.cs2n.org      NaN  SPIKE        
1771  NaN                       NaN  0            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           